Comparing Logistic regression with Transformers for Binary Email Classification(Ham,Spam)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("sms_spam")
print(dataset)

In [ ]:
print(dataset['train'][8])

In [ ]:
print(dataset['train'].features['label'])

#investigating TfidfVectorizer


In [ ]:
text = [
    "I won the lottery",
    "you won a lottery",
    "Congratulations! we are happy to offer you SDE-1 role at Amazon JFK office"
]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 500)
X = vectorizer.fit_transform(text)

print("shappe of TF-IDF matrix:", X.shape)
print("Example of feature names:", vectorizer.get_feature_names_out()[:20])


Vectorizing the dataset! getting ready for that prediction

In [ ]:
# doing the split with Hugging Face to avoid weird indexing issues
splits = dataset['train'].train_test_split(test_size=0.2, seed=42, stratify_by_column='label')

# pulling out the text and labels as plain lists (keeping it simple)
train_texts = list(splits['train']['sms'])
train_labels = list(map(int, splits['train']['label']))
test_texts  = list(splits['test']['sms'])
test_labels = list(map(int, splits['test']['label']))

# vectorizing with tf-idf — small cap on features to stay fast
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# fitting on train only, then applying to test
X_train = vectorizer.fit_transform(train_texts)
X_test  = vectorizer.transform(test_texts)

# sanity check — rows = docs, cols = vocab size
print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)

# quick peek at what words made it in
print("Example features:", vectorizer.get_feature_names_out()[:20])


In [ ]:
# training a simple baseline so I have a reference point
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score
import matplotlib.pyplot as plt

# fitting the model on the tf-idf features
clf = LogisticRegression(max_iter=2000, n_jobs=-1)  # cranking up max_iter just in case
clf.fit(X_train, train_labels)

# getting predictions on the test set to see how well it generalizes
test_pred = clf.predict(X_test)

# quick metrics to get the vibe (accuracy + macro-F1 for imbalance)
acc = accuracy_score(test_labels, test_pred)
f1  = f1_score(test_labels, test_pred, average='macro')
print("test accuracy:", acc)
print("macro-F1:", f1)
print("\nclassification report:\n", classification_report(test_labels, test_pred, digits=3, target_names=["ham","spam"]))

In [ ]:
# drawing the confusion matrix to see *how* it’s making mistakes
cm = confusion_matrix(test_labels, test_pred, labels=[0,1])  # 0=ham, 1=spam
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["ham","spam"])
disp.plot(values_format="d")
plt.title("TF-IDF + LogisticRegression — Confusion Matrix")
plt.show()

Now trying this with AI to compare which method performs better

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
example = "You won a lottery"

tokens = tokenizer.tokenize(example)
ids = tokenizer.convert_tokens_to_ids(tokens)

print("Tokens:", tokens)
print("IDs:", ids)


***Using DistilBERT for classification. BERT is great at understanding text and next word predicition through self attention***

I am going to try out a single training step to first set up the training process

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
#setting up tokenizer-same as model
tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 2,
).to(device)

In [ ]:
text = [
    "Hey, are we still meeting at 3pm today?",              # ham
    "WIN BIG! Claim your free prize now—limited offer!!!"   # spam"
]
labels = torch.tensor([0,1]).to(device)

enc = tok(
    text,
    truncation = True,
    padding = "max_length",
    max_length = 16,
    return_tensors = "pt"
).to(device)
# print("input_ids shape:", enc["input_ids"].shape)
# print("attention_mask shape:", enc["attention_mask"].shape)
# print("labels shape:", labels.shape)
print(enc)

In [ ]:
out = model(**enc, labels=labels)
loss = out.loss
logits = out.logits

print("loss:", float(loss))
print("logits shape:", logits.shape)
print("logits:", logits)


trying to calculate the loss myself to confirm that we get the same value

In [ ]:
import torch.nn.functional as F

# Cross-entropy expects raw logits and integer labels
ce = F.cross_entropy(logits, labels, reduction="mean")
print("CE re-computed:", float(ce.detach()))


now that we have the forward pass and loss, let’s do the backward pass and optimizer step — the moment where learning actually happens.

In [ ]:
from torch.optim import AdamW
#activate dropouts
model.train()
#creating optimizer with model parameters
opt = AdamW(model.parameters(),lr=2e-5)

# 3) forward pass to get loss again
out = model(**enc, labels=labels)
loss = out.loss
print("loss before:", float(loss))

# 4) zero old gradients
opt.zero_grad()

# 5) backward pass (calculate gradients)
loss.backward()

# 6) clip gradients (keeps them stable)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# 7) optimizer step (update weights)
opt.step()

we track the loss after this one trainining example

In [ ]:
import torch.nn.functional as F

# evaluate mode for a clean measurement (no dropout)
model.eval()
with torch.no_grad():
    out_after = model(**enc, labels=labels)
    loss_after = out_after.loss
    probs_after = F.softmax(out_after.logits, dim=-1)

print("loss after:", float(loss_after))
print("probs after:\n", probs_after.cpu().numpy())  # rows = examples, cols = [ham, spam]


the predictions are still ham. but the logits still shifted with the two examples which is great, it means with more data we can shift this to be more accurate

Now we are going to replicate this training loop but with a full dataset.

In [ ]:
from datasets import load_dataset

# load sms_spam dataset
dataset = load_dataset("sms_spam")

print(dataset)
print(dataset["train"][0])  # peek at the first example


In [ ]:
from datasets import ClassLabel
# stratified 80/20 split so class balance is preserved
splits = dataset["train"].train_test_split(
    test_size=0.2, seed=42, stratify_by_column="label"
)
train_ds = splits["train"]
val_ds   = splits["test"]

# quick sanity checks
print("train size:", len(train_ds), "val size:", len(val_ds))

# show label names if available (0=ham, 1=spam)
labels_feat = train_ds.features["label"]
if isinstance(labels_feat, ClassLabel):
    print("label names:", labels_feat.names)

# tiny peek at one of each split
print("train ex:", train_ds[0])
print("val ex:", val_ds[0])

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    texts = [x["sms"] for x in batch]
    labels = [x["label"] for x in batch]
    enc = tok(
        texts,
        truncation=True,
        padding="max_length",
        max_length=64,
        return_tensors="pt"
    )
    enc["labels"] = torch.tensor(labels, dtype=torch.long)
    return enc.to(device)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

# sanity check: grab one batch
batch = next(iter(train_loader))
print({k: v.shape for k, v in batch.items()})


mini train loop

In [ ]:
import torch
from torch.optim import AdamW
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification

# 1) model (distilbert + 2-class head) on the same device
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
).to(device)

# 2) optimizer
opt = AdamW(model.parameters(), lr=2e-5)

# 3) one short epoch over train data
model.train()
running_loss, correct, total = 0.0, 0, 0

for step, batch in enumerate(train_loader):
    # forward
    out = model(**batch)                 # batch has input_ids, attention_mask, labels
    loss = out.loss                      # scalar loss
    logits = out.logits                  # [B, 2]
    preds = logits.argmax(dim=-1)        # [B]

    # metrics
    running_loss += loss.item() * preds.size(0)
    correct += (preds == batch["labels"]).sum().item()
    total += preds.size(0)

    # backward
    opt.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    opt.step()

    # keep it short for now (just ~200 examples), then we evaluate
    if (step+1) * train_loader.batch_size >= 200:
        break

train_loss = running_loss / total
train_acc  = correct / total
print(f"train: n={total}, loss={train_loss:.4f}, acc={train_acc:.3f}")

# 4) quick validation pass
model.eval()
val_correct, val_total, val_loss_sum = 0, 0, 0.0
with torch.no_grad():
    for batch in val_loader:
        out = model(**batch)
        val_loss_sum += out.loss.item() * batch["labels"].size(0)
        preds = out.logits.argmax(dim=-1)
        val_correct += (preds == batch["labels"]).sum().item()
        val_total += batch["labels"].size(0)

val_loss = val_loss_sum / val_total
val_acc  = val_correct / val_total
print(f"val:   n={val_total}, loss={val_loss:.4f}, acc={val_acc:.3f}")


In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
import torch
import torch.nn.functional as F

# fresh model (optional). comment this line out if you want to continue training the same model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

opt = AdamW(model.parameters(), lr=2e-5)
# simple linear warmdown scheduler (keeps things stable)
scheduler = LinearLR(opt, start_factor=1.0, end_factor=0.1, total_iters=3)  # 3 epochs

def evaluate():
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    with torch.no_grad():
        for batch in val_loader:
            out = model(**batch)
            loss_sum += out.loss.item() * batch["labels"].size(0)
            preds = out.logits.argmax(dim=-1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)
    return loss_sum/total, correct/total

EPOCHS = 10
for epoch in range(1, EPOCHS+1):
    model.train()
    total, correct, loss_sum = 0, 0, 0.0
    for batch in train_loader:
        out = model(**batch)
        loss = out.loss

        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        # stats
        preds = out.logits.argmax(dim=-1)
        loss_sum += loss.item() * preds.size(0)
        correct += (preds == batch["labels"]).sum().item()
        total += preds.size(0)

    train_loss, train_acc = loss_sum/total, correct/total
    val_loss, val_acc = evaluate()
    scheduler.step()

    print(f"epoch {epoch}: train loss {train_loss:.4f} acc {train_acc:.3f} | val loss {val_loss:.4f} acc {val_acc:.3f}")


The loss has significantly reduced now with the 10 epochs i run from the 0.5 to 0.05 which is hige on acuracy. it is getting to 99 percent accuracy also which is already better than logistic regression

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch

# collect all predictions on the validation set
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        out = model(**batch)
        preds = out.logits.argmax(dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch["labels"].cpu().tolist())

cm = confusion_matrix(all_labels, all_preds, labels=[0,1])  # 0=ham, 1=spam
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["ham","spam"])
disp.plot(values_format="d")
plt.title("Ham/Spam — Confusion Matrix")
plt.show()

print("Confusion matrix:\n", cm)


In [ ]:
# save
save_dir = "distilbert-hamspam-manual"
model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)
print("saved to:", save_dir)

# load later (new objects)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tok2 = AutoTokenizer.from_pretrained(save_dir)
model2 = AutoModelForSequenceClassification.from_pretrained(save_dir).to(device)

# quick sanity check on a sample
import torch.nn.functional as F
text = "WIN BIG! Claim your prize now!"
enc = tok2(text, return_tensors="pt", truncation=True, padding=True, max_length=64).to(device)
with torch.no_grad():
    probs = F.softmax(model2(**enc).logits, dim=-1).cpu().numpy()[0]
print("probs [ham, spam]:", probs)


In [ ]:
# 1) Make sure the hub library is up to date
%pip install -U huggingface_hub

# 2) Clear any cached/incorrect credentials (optional but helpful)
from huggingface_hub import logout
logout()  # ignores if you weren't logged in

# 3) Login with an access token (NOT your password)
from huggingface_hub import notebook_login
notebook_login()  # paste your HF token here (Settings -> Access Tokens -> New token with "Write" permission)


In [ ]:
from huggingface_hub import whoami
whoami()


In [ ]:
from huggingface_hub import create_repo, upload_folder

repo_id = "AgnessLungu/distilbert-hamspam"     # you can rename 'distilbert-hamspam' if you prefer
create_repo(repo_id, repo_type="model", exist_ok=True)

# IMPORTANT: this path must match the folder you saved earlier (the one with config.json, tokenizer.json, model.safetensors/bin)
folder_path = "distilbert-hamspam-manual"      # <-- change if your saved folder name is different

upload_folder(
    repo_id=repo_id,
    folder_path=folder_path,
    commit_message="Initial model upload from Colab"
)

print("✅ Pushed to:", f"https://huggingface.co/{repo_id}")


In [ ]:
from huggingface_hub import create_repo, upload_folder
import os

space_id = "AgnessLungu/ham-spam-ui"           # keep or rename
model_id = "AgnessLungu/distilbert-hamspam"    # your model repo

# (Re)create the local app files if needed
os.makedirs("space_app", exist_ok=True)

app_py = f"""
import gradio as gr
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_ID = "{model_id}"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID).to(DEVICE)
model.eval()

labels = getattr(model.config, "id2label", {{0: "ham", 1: "spam"}})

def predict(text):
    if not text or not text.strip():
        return {{labels.get(0, "ham"): 1.0, labels.get(1, "spam"): 0.0}}
    enc = tok(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(DEVICE)
    with torch.no_grad():
        logits = model(**enc).logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()[0]
    return {{labels[0]: float(probs[0]), labels[1]: float(probs[1])}}

with gr.Blocks(title="Ham vs Spam") as demo:
    gr.Markdown("# Ham vs Spam\\nPaste an email and click **Classify**.")
    inp = gr.Textbox(lines=8, label="Email text")
    out = gr.Label(label="Prediction (ham/spam)")
    gr.Button("Classify").click(fn=predict, inputs=inp, outputs=out)

if __name__ == "__main__":
    demo.launch()
"""
with open("space_app/app.py", "w") as f:
    f.write(app_py)

with open("space_app/requirements.txt", "w") as f:
    f.write("transformers>=4.40\ntorch\ngradio>=4.0\n")

# 1) Create the Space (repo_type='space' + sdk='gradio')
create_repo(space_id, repo_type="space", space_sdk="gradio", exist_ok=True)

# 2) Upload the folder to the Space (IMPORTANT: repo_type='space')
upload_folder(
    repo_id=space_id,
    repo_type="space",                   # <-- this fixes the 404
    folder_path="space_app",
    commit_message="Initial UI"
)

print("✅ Space created/updated:", f"https://huggingface.co/spaces/{space_id}")


In [ ]:
# Define human-readable labels
id2label = {0: "ham", 1: "spam"}
label2id = {"ham": 0, "spam": 1}

model.config.id2label = id2label
model.config.label2id = label2id

# save again with these mappings
model.save_pretrained("distilbert-hamspam-manual")
tok.save_pretrained("distilbert-hamspam-manual")


In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="AgnessLungu/distilbert-hamspam",
    folder_path="distilbert-hamspam-manual",
    commit_message="Add ham/spam labels"
)


In [ ]:
# run this once in your training notebook, then re-upload the folder
id2label = {0: "ham", 1: "spam"}
label2id = {"ham": 0, "spam": 1}

model.config.id2label = id2label
model.config.label2id = label2id

model.save_pretrained("distilbert-hamspam-manual")
tok.save_pretrained("distilbert-hamspam-manual")

from huggingface_hub import upload_folder
upload_folder(
    repo_id="AgnessLungu/distilbert-hamspam",
    folder_path="distilbert-hamspam-manual",
    commit_message="Add id2label/label2id (ham/spam)"
)


In [ ]:
from huggingface_hub import upload_folder
upload_folder(
    repo_id="AgnessLungu/ham-spam-ui",
    repo_type="space",
    folder_path="space_app",
    commit_message="Show human-readable labels and confidence"
)
